## Imports

In [1]:
# autoreload import module on change (does not work with from x import y)
%load_ext autoreload
%autoreload 2

In [2]:
# Import functions
import pandas as pd
import numpy as np
from pathlib import Path
from mimic_constants import *
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, f1_score, matthews_corrcoef

In [ ]:
def compute_metrics(y_true, y_pred, y_prob):
    """Compute various classification metrics for binary classification."""
    # y_true: Actual binary labels (0 or 1)
    # y_pred: Predicted binary labels (0 or 1), thresholding applied
    # y_prob: Raw probabilities for the positive class

    auc = roc_auc_score(y_true, y_prob)
    avg_precision = average_precision_score(y_true, y_prob)
    
    # Convert y_pred to binary predictions if it's probabilities
    y_pred_binary = (y_pred > 0.5).astype(int)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred_binary)
    
    # Metrics calculations
    tpr = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0  # Sensitivity, Recall
    tnr = cm[0, 0] / (cm[0, 0] + cm[0, 1]) if (cm[0, 0] + cm[0, 1]) > 0 else 0  # Specificity
    ppv = cm[1, 1] / (cm[1, 1] + cm[0, 1]) if (cm[1, 1] + cm[0, 1]) > 0 else 0  # Precision
    npv = cm[0, 0] / (cm[0, 0] + cm[1, 0]) if (cm[0, 0] + cm[1, 0]) > 0 else 0  # Negative Predictive Value
    f1 = f1_score(y_true, y_pred_binary)
    mcc = matthews_corrcoef(y_true, y_pred_binary)

    return {
        'AUC': auc,
        'Average Precision': avg_precision,
        'TPR': tpr,
        'TNR': tnr,
        'PPV': ppv,
        'NPV': npv,
        'F1': f1,
        'MCC': mcc
    }

In [4]:
# Import cleaned master dataframe
df = get_master_df(idp=True)

## Prep data for Cardiomegaly

In [5]:
label = 'Cardiomegaly'
f'Number of Total Samples: {len(df)}'

'Number of Total Samples: 2662'

In [6]:
df['age_label'] = df['anchor_age'].apply(lambda x: min(x / 100, 1))
demographic_cols = ['age_label']

In [7]:
X = df[demographic_cols + significant_variables[1:]]
Y = df[[label]]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y[label])

In [9]:
# 2. Mean Imputation for NaNs in X_train
imputer = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Ensure that the train and test sets have the same columns after encoding
X_test_encoded = X_test_imputed.reindex(columns=X_test_imputed.columns, fill_value=0)

X_train_encoded = sm.add_constant(X_train_imputed)  # Add intercept term
X_test_encoded = sm.add_constant(X_test_encoded)

X_train_encoded = X_train_encoded.reset_index(drop=True)
X_test_encoded = X_test_encoded.reset_index(drop=True)
Y_train = Y_train.reset_index(drop=True)
Y_test = Y_test.reset_index(drop=True)

## Logistic Regression

In [10]:
# 4. Logistic Regression using Statsmodels
logit_model = sm.Logit(Y_train, X_train_encoded)
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.567988
         Iterations 6


In [11]:
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
===================================================================
Model:                Logit            Method:           MLE       
Dependent Variable:   Cardiomegaly     Pseudo R-squared: 0.072     
Date:                 2024-09-08 19:31 AIC:              2436.4942 
No. Observations:     2129             BIC:              2487.4649 
Df Model:             8                Log-Likelihood:   -1209.2   
Df Residuals:         2120             LL-Null:          -1302.5   
Converged:            1.0000           LLR p-value:      4.5397e-36
No. Iterations:       6.0000           Scale:            1.0000    
-------------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-------------------------------------------------------------------
const               -1.3124   1.4463 -0.9074 0.3642 -4.1470  1.5222
age_label            2.5082   0.3372  7.4390 0.0000  1.8474  3.1691
Chloride_mean       -0.0252   0.0112 -2.2554 0.0241 -0.0471 -0.0033
RR_mean              0.0051   0.0098  0.5274 0.5979 -0.0140  0.0243
Urea_Nitrogren_mean  0.0143   0.0038  3.7097 0.0002  0.0067  0.0218
Magnesium_mean       0.6784   0.2297  2.9533 0.0031  0.2282  1.1285
Glucose_mean         0.0009   0.0013  0.6698 0.5030 -0.0017  0.0035
Phosphate_mean       0.1706   0.0732  2.3325 0.0197  0.0273  0.3140
Hematocrit_mean      0.0162   0.0094  1.7287 0.0839 -0.0022  0.0346
===================================================================

"""

In [12]:
# Z-scores of the coefficients
z_scores = pd.concat([result.summary2().tables[1]['z'], result.summary2().tables[1]['P>|z|']], axis=1)

# Predictions and evaluations
Y_test_pred_prob = result.predict(X_test_encoded)
Y_test_pred = (Y_test_pred_prob > 0.5).astype(int)

auc_score = roc_auc_score(Y_test, Y_test_pred_prob)
f1 = f1_score(Y_test, Y_test_pred)
conf_matrix = confusion_matrix(Y_test, Y_test_pred)

In [15]:
df_fold_metrics = pd.DataFrame({'test_metrics': compute_metrics(Y_test, Y_test_pred, Y_test_pred_prob)})
test_metrics_df = df_fold_metrics['test_metrics'].apply(pd.Series).add_prefix('test_')

In [17]:
test_metrics_df.T.reset_index(drop=True)

,AUC,Average Precision,TPR,TNR,PPV,NPV,F1,MCC
0,0.6993,0.8260,0.9464,0.1688,0.7263,0.5745,0.8219,0.1861


In [13]:
# Outputs
print(f'Confusion Matrix:\n{conf_matrix}', sep='\n')
pd.set_option('display.float_format', '{:.4f}'.format)

AUC: 0.6993130026809651
F1 Score: 0.8218859138533178
Confusion Matrix:
[[ 27 133]
 [ 20 353]]


In [14]:
z_scores.sort_values(by='P>|z|', ascending=True).head(20)

,z,P>|z|
age_label,7.4390,0.0000
Urea_Nitrogren_mean,3.7097,0.0002
Magnesium_mean,2.9533,0.0031
Phosphate_mean,2.3325,0.0197
Chloride_mean,-2.2554,0.0241
Hematocrit_mean,1.7287,0.0839
const,-0.9074,0.3642
Glucose_mean,0.6698,0.5030
RR_mean,0.5274,0.5979


In [31]:
# Import functions
import pandas as pd
import numpy as np
from pathlib import Path
from mimic_constants import *
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, f1_score, matthews_corrcoef
from scipy import stats  # Add this import

def compute_metrics(y_true, y_pred, y_prob):
    """Compute various classification metrics for binary classification."""
    auc = roc_auc_score(y_true, y_prob)
    avg_precision = average_precision_score(y_true, y_prob)
    
    y_pred_binary = (y_pred > 0.5).astype(int)
    cm = confusion_matrix(y_true, y_pred_binary)
    
    tpr = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0
    tnr = cm[0, 0] / (cm[0, 0] + cm[0, 1]) if (cm[0, 0] + cm[0, 1]) > 0 else 0
    ppv = cm[1, 1] / (cm[1, 1] + cm[0, 1]) if (cm[1, 1] + cm[0, 1]) > 0 else 0
    npv = cm[0, 0] / (cm[0, 0] + cm[1, 0]) if (cm[0, 0] + cm[1, 0]) > 0 else 0
    f1 = f1_score(y_true, y_pred_binary)
    mcc = matthews_corrcoef(y_true, y_pred_binary)

    return {
        'AUC': auc,
        'Average Precision': avg_precision,
        'TPR': tpr,
        'TNR': tnr,
        'PPV': ppv,
        'NPV': npv,
        'F1': f1,
        'MCC': mcc
    }

# Import cleaned master dataframe
df = get_master_df(idp=True)

# Prep data for Cardiomegaly
label = 'Cardiomegaly'
df['age_label'] = df['anchor_age'].apply(lambda x: min(x / 100, 1))
demographic_cols = ['age_label']

X = df[demographic_cols + significant_variables[1:]]
Y = df[[label]]

_X, X_test, _Y, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y[label])

# Initialize 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize lists to store results
fold_metrics = []
fold_coefficients = []
best_fold, best_mcc, best_z_scores = (0, 0, None)
for fold, (train_index, val_index) in enumerate(kf.split(_X), 1):
    X_train, X_val = _X.iloc[train_index], _X.iloc[val_index]
    Y_train, Y_val = _Y.iloc[train_index], _Y.iloc[val_index]

    # Mean Imputation for NaNs
    imputer = SimpleImputer(strategy='mean')
    X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
    X_val_imputed = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)

    # Add constant term
    X_train_encoded = sm.add_constant(X_train_imputed)
    X_val_encoded = sm.add_constant(X_val_imputed)

    # Ensure that the train and test sets have the same columns after encoding
    X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
    X_test_encoded = X_test_imputed.reindex(columns=X_test_imputed.columns, fill_value=0)
    X_test_encoded = sm.add_constant(X_test_encoded)

    X_train_encoded = X_train_encoded.reset_index(drop=True)
    Y_train = Y_train.reset_index(drop=True)
    X_val_encoded = X_val_encoded.reset_index(drop=True)
    Y_val = Y_val.reset_index(drop=True)
    X_test_encoded = X_test_encoded.reset_index(drop=True)
    Y_test = Y_test.reset_index(drop=True)

    # Fit logistic regression
    logit_model = sm.Logit(Y_train, X_train_encoded)
    result = logit_model.fit()

    # Store coefficients
    fold_coefficients.append(result.params)

    # Predictions and evaluations
    Y_val_pred_prob = result.predict(X_val_encoded)
    Y_val_pred = (Y_val_pred_prob > 0.5).astype(int)

    # Compute and store metrics
    val_metrics = compute_metrics(Y_val, Y_val_pred, Y_val_pred_prob)

    # Predictions and evaluations
    Y_test_pred_prob = result.predict(X_test_encoded)
    Y_test_pred = (Y_test_pred_prob > 0.5).astype(int)

    # Compute and store metrics
    metrics = compute_metrics(Y_test, Y_test_pred, Y_test_pred_prob)
    fold_metrics.append(metrics)

    if val_metrics['MCC'] > best_mcc:
        best_mcc = val_metrics['MCC']
        best_z_scores = pd.concat([result.summary2().tables[1]['z'], result.summary2().tables[1]['P>|z|']], axis=1)
        best_fold = fold

    print(f"Fold {fold} completed")

Optimization terminated successfully.
         Current function value: 0.573359
         Iterations 6
Fold 1 completed
Optimization terminated successfully.
         Current function value: 0.569345
         Iterations 6
Fold 2 completed
Optimization terminated successfully.
         Current function value: 0.571728
         Iterations 6
Fold 3 completed
Optimization terminated successfully.
         Current function value: 0.568088
         Iterations 6
Fold 4 completed
Optimization terminated successfully.
         Current function value: 0.565717
         Iterations 6
Fold 5 completed
Optimization terminated successfully.
         Current function value: 0.566485
         Iterations 6
Fold 6 completed
Optimization terminated successfully.
         Current function value: 0.563554
         Iterations 6
Fold 7 completed
Optimization terminated successfully.
         Current function value: 0.564759
         Iterations 6
Fold 8 completed
Optimization terminated successfully.
         C

In [33]:
# Compute average metrics across folds
avg_metrics = pd.DataFrame(fold_metrics).mean()
std_metrics = pd.DataFrame(fold_metrics).std()
print("\nAverage Metrics across 10 folds:")
print(pd.concat([avg_metrics, std_metrics], axis=1))

# Compute average coefficients across folds
print(f"\nCoefficients for best fold {best_fold} (Val MCC: {best_mcc}):")
print(best_z_scores.sort_values(by='P>|z|', ascending=True).head(20))


Average Metrics across 10 folds:
                       0      1
AUC               0.6985 0.0019
Average Precision 0.8257 0.0018
TPR               0.9485 0.0028
TNR               0.1700 0.0105
PPV               0.7271 0.0022
NPV               0.5861 0.0116
F1                0.8232 0.0012
MCC               0.1926 0.0107

Coefficients for best fold 9 (Val MCC: 0.33876828519289826):
                          z  P>|z|
age_label            6.8376 0.0000
Urea_Nitrogren_mean  3.3646 0.0008
Magnesium_mean       2.7827 0.0054
Chloride_mean       -2.4440 0.0145
Phosphate_mean       2.0500 0.0404
Hematocrit_mean      1.5601 0.1187
RR_mean              0.4735 0.6358
const               -0.4335 0.6647
Glucose_mean         0.3394 0.7343
